# Convert into npz

In [18]:
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

AUDIO_DIR = 'audio_files/fma_small'

In [20]:
def get_tids_from_directory(audio_dir):
    """Get track IDs from the mp3s in a directory.
    Parameters
    ----------
    audio_dir : str
        Path to the directory where the audio files are stored.
    Returns
    -------
        A list of track IDs.
    """
    tids = []
    for _, dirnames, files in os.walk(audio_dir):
        if dirnames == []:
            tids.extend(int(file[:-4]) for file in files)
    return tids


def get_audio_path(audio_dir, track_id):
    """
    Return the path to the mp3 given the directory where the audio is stored
    and the track ID.
    Examples
    """
    tid_str = '{:06d}'.format(track_id)
    a = os.path.join(audio_dir, tid_str[:3], tid_str + '.mp3')
    a = a.replace("\\", "/")
    return a

def create_spectogram(track_id):
    filename = get_audio_path(AUDIO_DIR, track_id)
    y, sr = librosa.load(filename)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=1024)
    spect = librosa.power_to_db(spect, ref=np.max)
    return spect.T

def plot_spect(track_id):
    spect = create_spectogram(track_id)
    print(spect.shape)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spect.T, y_axis='mel', fmax=8000, x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.show()
    
def create_array(df):
    genres = []
    X_spect = np.empty((0, 640, 128))
    count = 0
    #Code skips records in case of errors
    for index, row in df.iterrows():
        try:
            count += 1
            track_id = int(row['track_id'])
            genre = str(row[('track', 'genre_top')])
            spect = create_spectogram(track_id)

            # Normalize for small shape differences
            spect = spect[:640, :]
            X_spect = np.append(X_spect, [spect], axis=0)
            genres.append(dict_genres[genre])
            if count % 100 == 0:
                print("Currently processing: ", count)
        except:
            print("Couldn't process: ", count)
            continue
    y_arr = np.array(genres)
    return X_spect, y_arr    

def splitDataFrameIntoSmaller(df, chunkSize = 1600): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]


In [3]:
tids = get_tids_from_directory(AUDIO_DIR)

## Load dataset with genre and track IDs


In [26]:
filepath = 'fma_metadata/tracks.csv'
tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])
keep_cols = [('set', 'split'),
('set', 'subset'),('track', 'genre_top')]

df_all = tracks[keep_cols]
df_all = df_all[df_all[('set', 'subset')] == 'small']

df_all['track_id'] = df_all.index
df_all.head()


set            track track_id
             split subset genre_top         
track_id                                    
2         training  small   Hip-Hop        2
5         training  small   Hip-Hop        5
10        training  small       Pop       10
140       training  small      Folk      140
141       training  small      Folk      141

In [29]:
dict_genres = {'Electronic':1, 'Experimental':2, 'Folk':3, 'Hip-Hop':4, 
               'Instrumental':5,'International':6, 'Pop' :7, 'Rock': 8  }


## Create Arrays


In [31]:
df_all[('set', 'split')].unique()


array(['training', 'validation', 'test'], dtype=object)

## Create train, validation and test subsets¶


In [5]:
df_train = df_all[df_all[('set', 'split')]=='training']
df_valid = df_all[df_all[('set', 'split')]=='validation']
df_test = df_all[df_all[('set', 'split')]=='test']

df_test.to_csv("test_file")
df_test_ = pd.read_csv("test_file")

np.savez('test_arr', X_test, y_test)
np.savez('valid_arr', X_valid, y_valid)


### Note: Train data split into 4 chunks to do the slow pre-processing in phases

In [6]:
listDf = splitDataFrameIntoSmaller(df_train)
df1_train = listDf[0]
df2_train = listDf[1]
df3_train = listDf[2]
df4_train = listDf[3]


In [7]:
X_train1, y_train1 = create_array(df1_train)
np.savez('train1_arr', X_train1, y_train1)  

X_train2, y_train2 = create_array(df2_train)
np.savez('train2_arr', X_train2, y_train2)

X_train3, y_train3 = create_array(df3_train)
np.savez('train3_arr', X_train3, y_train3)

X_train4, y_train4 = create_array(df4_train)
np.savez('train4_arr', X_train4, y_train4)


## Concatenate and Shuffle data

In [8]:
npzfile = np.load('train1_arr.npz')
X_train1 = npzfile['arr_0']
y_train1 = npzfile['arr_1']

npzfile = np.load('train2_arr.npz')
X_train2 = npzfile['arr_0']
y_train2 = npzfile['arr_1']

npzfile = np.load('train3_arr.npz')
X_train3 = npzfile['arr_0']
y_train3 = npzfile['arr_1']

npzfile = np.load('train4_arr.npz')
X_train4 = npzfile['arr_0']
y_train4 = npzfile['arr_1']

npzfile = np.load('valid_arr.npz')
X_valid = npzfile['arr_0']
y_valid = npzfile['arr_1']


## Concatenate train data

In [17]:
y_train = np.concatenate((y_train1, y_train2, y_train3, y_train4), axis = 0)

In [10]:
y_train = y_train -1
y_valid = y_valid -1

np.save("y_train", y_train)
np.save("y_valid", y_valid)

a = librosa.core.db_to_power(X_train1, ref = 1.0)

X_train_raw2 = librosa.core.db_to_power(X_train2, ref = 1.0)
X_train_raw3 = librosa.core.db_to_power(X_train3, ref = 1.0)
X_train_raw4 = librosa.core.db_to_power(X_train4, ref = 1.0)
X_train_raw = np.concatenate((a, X_train_raw2), axis = 0)

np.savez('X_train_raw', X_train_raw)
X_train_raw = np.concatenate((a, X_train_raw2), axis = 0)
np.save("X_train_raw3", X_train_raw3 )
np.save("X_train_raw4", X_train_raw4)

In [11]:
npzfile = np.load('X_train_raw.npz')
x_train_raw1 = npzfile['arr_0']

X_train_log_ = np.log(x_train_raw1)
np.save("X_train_log_3200", X_train_log_) 

X_train_log = np.load('X_train_log.npy')

In [36]:
X_train_raw_ = np.concatenate((X_train_log, X_train_log_), axis = 0)
np.savez("Xtrain_123_4800", X_train_raw_)

X_train_raw4 = np.load("X_train_raw4.npy")
X_train_log1 = np.log(X_train_raw4)
np.save("X_train_log1", X_train_log1)


In [41]:
#X_valid_raw = librosa.core.db_to_power(X_valid, ref=1.0)

X_train_log1 = np.load("X_train_log1.npy")

npzfile = np.load('Xtrain_123_4800.npz')
Xtrain_123_4800 = npzfile['arr_0']

X_train_log = np.concatenate((Xtrain_123_4800 , X_train_log1), axis = 0)
np.savez("X_train_log", X_train_log)

npzfile = np.load('X_train_log.npz')
x_train_log = npzfile['arr_0']

x_valid_raw = librosa.core.db_to_power(X_valid, ref = 1.0)
x_valid_log = np.log(x_valid_raw)
np.save("x_valid_log", x_valid_log)

x_valid_log = np.load("x_valid_log.npy")

y_train = np.load("y_train.npy")
y_valid = np.load("y_valid.npy")

In [6]:
X_train, y_train = unison_shuffled_copies(x_train_log, y_train)
X_valid, y_valid = unison_shuffled_copies(x_valid_log, y_valid)

np.savez('shuffled_train', X_train, y_train)
np.savez('shuffled_valid', X_valid, y_valid)